In [2]:
# import pandas as pd
import polars as pl
import plotly.express as px

In [6]:
# get data
# df = pd.read_csv("/workspaces/datadex/data/fy24_aop_schedule.csv")
df = pl.read_csv("/workspaces/datadex/data/fy24_aop_schedule.csv", try_parse_dates=True)

# df.start_date = pd.to_datetime(df.start_date)
# df = df.with_columns(
#     pl.col('start_date').str.to_date('%Y-%m-%d'),
# )
df

process,strip,block,seam,ws,key,start_date,end_date,format_date,year,fy,month_num,rom_qty,pmath_ws,strip_block,rom_ash_ad,rom_ash_ar,rom_tm_ar,rom_tm_ad,rom_ts_ar,rom_ts_ad,wash_ash_ad,wash_ts_ad,wash_tm_ar,wash_yield,wash_k2o_ad,wash_na2o_ad,wash_alk_ad,wash_qty
str,str,i64,str,str,str,datetime[μs],datetime[μs],datetime[μs],i64,str,i64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Coal""","""S40""",9,"""PF1B""","""PF1B""","""40|9|PF1B""",2023-02-01 07:00:00,2023-02-01 18:30:00,2023-02-01 07:00:00,2023,"""FY2023""",2,10124.19,"""PF1""","""S40/09""",0.334087,0.31895,0.102172,0.059561,0.003229,0.003382,0.117173,0.003847,0.05956,0.606377,0.005469,0.010035,0.0120714,6139.076013
"""Coal""","""S39""",4,"""BY1A2""","""BY1A2-BY2A""","""39|4|BY1A2-BY2…",2023-02-01 07:00:00,2023-02-01 16:03:00,2023-02-01 07:00:00,2023,"""FY2023""",2,13167.16,"""BY1""","""S39/04""",0.22894,0.215437,0.108073,0.05217,0.002322,0.002468,0.183212,0.002368,0.053655,0.844277,0.012997,0.006573,0.0173224,11116.72864
"""Coal""","""S39""",3,"""BY1A2""","""BY1A2-BY2A""","""39|3|BY1A2-BY2…",2023-02-01 16:03:00,2023-02-01 19:00:00,2023-02-01 16:03:00,2023,"""FY2023""",2,4287.9,"""BY1""","""S39/03""",0.248475,0.233833,0.107171,0.051267,0.002322,0.002467,0.185992,0.00236,0.05229,0.826465,0.013109,0.006613,0.0174609,3543.800438
"""Coal""","""S40""",8,"""PF1B""","""PF1B""","""40|8|PF1B""",2023-02-01 18:30:00,2023-02-01 19:00:00,2023-02-01 18:30:00,2023,"""FY2023""",2,438.34,"""PF1""","""S40/08""",0.32238,0.307768,0.102527,0.059917,0.003205,0.003358,0.123587,0.003793,0.058106,0.632195,0.006262,0.009912,0.0127847,277.119416
"""Coal""","""S40""",8,"""PF1B""","""PF1B""","""40|8|PF1B""",2023-02-01 19:00:00,2023-02-02 06:24:00,2023-02-01 19:00:00,2023,"""FY2023""",2,10039.7,"""PF1""","""S40/08""",0.32238,0.307768,0.102527,0.059917,0.003205,0.003358,0.123587,0.003793,0.058106,0.632195,0.006262,0.009912,0.0127847,6347.053064
"""Coal""","""S39""",3,"""BY1A2""","""BY1A2-BY2A""","""39|3|BY1A2-BY2…",2023-02-01 19:00:00,2023-02-02 07:00:00,2023-02-01 19:00:00,2023,"""FY2023""",2,17455.07,"""BY1""","""S39/03""",0.248475,0.233833,0.107171,0.051267,0.002322,0.002467,0.185992,0.00236,0.05229,0.826465,0.013109,0.006613,0.0174609,14426.00452
"""Coal""","""S40""",7,"""PF1B""","""PF1B""","""40|7|PF1B""",2023-02-02 06:24:00,2023-02-02 07:00:00,2023-02-02 06:24:00,2023,"""FY2023""",2,522.84,"""PF1""","""S40/07""",0.316337,0.302006,0.10207,0.05946,0.003203,0.003355,0.127196,0.003792,0.056941,0.641129,0.007317,0.009831,0.0137852,335.205118
"""Coal""","""S40""",7,"""PF1B""","""PF1B""","""40|7|PF1B""",2023-02-02 07:00:00,2023-02-02 15:10:00,2023-02-02 07:00:00,2023,"""FY2023""",2,7199.21,"""PF1""","""S40/07""",0.316337,0.302006,0.10207,0.05946,0.003203,0.003355,0.127196,0.003792,0.056941,0.641129,0.007317,0.009831,0.0137852,4615.621294
"""Coal""","""S39""",3,"""BY1A2""","""BY1A2-BY2A""","""39|3|BY1A2-BY2…",2023-02-02 07:00:00,2023-02-02 13:12:00,2023-02-02 07:00:00,2023,"""FY2023""",2,9019.8,"""BY1""","""S39/03""",0.248475,0.233833,0.107171,0.051267,0.002322,0.002467,0.185992,0.00236,0.05229,0.826465,0.013109,0.006613,0.0174609,7454.550678


In [7]:
# df.pmath_ws.drop_duplicates()
df.select("pmath_ws").unique()

pmath_ws
str
"""WW3"""
"""PF1"""
"""BY1"""
"""WW2"""
"""PF2"""
"""VA2"""
"""VA1"""
"""MA1"""
"""BR1"""


In [147]:
# Resource level aggregations
# todo up to here
# weighted av function
def weighted_average(df, value, weight):
    try:
        return (df[value] * df[weight]).sum() / df[weight].sum()
    except ZeroDivisionError:
        return 0


#  add HCV flag on wash_ash < 0.15
df["coal_type"] = df["wash_ash_ad"].apply(lambda x: "High CV" if x < 0.15 else "Low CV")

# change coal type to Low CV if pmath_ws is WN1
df.loc[df["pmath_ws"] == "WN1", "coal_type"] = "Low CV"

df["pmath_ws_raw"] = df["pmath_ws"]

# clean up so that any if pmath_ws is WW1 or WW2 and has  wash_alk_ad < 0.006  we rewrite wash_alk_ad to 0.0115
df.loc[
    (df["pmath_ws"].isin(["WW1", "WW2"])) & (df["wash_alk_ad"] < 0.006), "wash_alk_ad"
] = 0.0115

df_hcv = df.query('coal_type == "High CV"')

df

,process,strip,block,seam,ws,key,start_date,end_date,format_date,year,...,wash_ash_ad,wash_ts_ad,wash_tm_ar,wash_yield,wash_k2o_ad,wash_na2o_ad,wash_alk_ad,wash_qty,coal_type,pmath_ws_raw
0,Coal,S40,9,PF1B,PF1B,40|9|PF1B,2023-02-01 07:00:00,2023-02-01 18:30,2023-02-01 07:00,2023,...,0.117173,0.003847,0.059560,0.606377,0.005469,0.010035,0.012071,6139.076013,High CV,PF1
1,Coal,S39,4,BY1A2,BY1A2-BY2A,39|4|BY1A2-BY2A,2023-02-01 07:00:00,2023-02-01 16:03,2023-02-01 07:00,2023,...,0.183212,0.002368,0.053655,0.844277,0.012997,0.006573,0.017322,11116.728640,Low CV,BY1
2,Coal,S39,3,BY1A2,BY1A2-BY2A,39|3|BY1A2-BY2A,2023-02-01 16:03:00,2023-02-01 19:00,2023-02-01 16:03,2023,...,0.185992,0.002360,0.052290,0.826465,0.013109,0.006613,0.017461,3543.800438,Low CV,BY1
3,Coal,S40,8,PF1B,PF1B,40|8|PF1B,2023-02-01 18:30:00,2023-02-01 19:00,2023-02-01 18:30,2023,...,0.123587,0.003793,0.058106,0.632195,0.006262,0.009912,0.012785,277.119416,High CV,PF1
4,Coal,S40,8,PF1B,PF1B,40|8|PF1B,2023-02-01 19:00:00,2023-02-02 06:24,2023-02-01 19:00,2023,...,0.123587,0.003793,0.058106,0.632195,0.006262,0.009912,0.012785,6347.053064,High CV,PF1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,Coal,S52,28,VA1C1,VA1C1-VA2C,52|28|VA1C1-VA2C,2028-12-31 19:41:00,2028-12-31 23:10,2028-12-31 19:41,2028,...,0.097038,0.004330,0.037960,0.765149,0.007848,0.018423,0.019970,2527.680466,High CV,VA12
11610,Coal,S52,28,VA1C1,VA1C1-VA2C,52|28|VA1C1-VA2C,2028-12-31 23:10:00,2028-12-31 23:10,2028-12-31 23:10,2028,...,0.097038,0.004330,0.037960,0.765149,0.007848,0.018423,0.019970,0.007961,High CV,VA12
11611,Coal,S53,13,WW2C,WW2C,53|13|WW2C,2028-12-31 23:10:00,2028-12-31 23:59,2028-12-31 23:10,2028,...,0.138543,0.004283,0.062383,0.840888,0.006242,0.006794,0.010713,642.470271,High CV,WW2
11612,Coal,S52,28,VA1C1,VA1C1-VA2C,52|28|VA1C1-VA2C,2028-12-31 23:10:00,2028-12-31 23:59,2028-12-31 23:10,2028,...,0.097038,0.004330,0.037960,0.765149,0.007848,0.018423,0.019970,588.641611,High CV,VA12


In [148]:
# group by strip & weighted av of wash_alk_ad
df_strip = (
    df.groupby("strip")
    .apply(weighted_average, "wash_alk_ad", "wash_qty")
    .reset_index(name="wash_alk_ad")
    .query('strip not in ("S52","S53","S54","S55")')
)
df_strip
fig = px.bar(
    df_strip,
    x="strip",
    y="wash_alk_ad",
    title="Wash Alkalinity by Strip",
    template="presentation",
)
fig.add_hline(y=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    yaxis_tickformat=".1%",  # None
    # , yaxis_rangemode = 'tozero'
    xaxis_title=None,
    yaxis_title="Wash Alkalinity",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
)

fig.update_layout(width=1000, height=600)
fig

In [149]:
# group by strip & weighted av of wash_alk_ad
df_strip = (
    df.groupby("fy")
    .apply(weighted_average, "wash_alk_ad", "wash_qty")
    .reset_index(name="wash_alk_ad")
    # .query('strip not in ("S52","S53","S54","S55")')
)
df_strip
fig = px.bar(
    df_strip,
    x="fy",
    y="wash_alk_ad",
    title="Wash Alkalinity by FY",
    template="presentation",
)
fig.add_hline(y=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    yaxis_tickformat=".1%",  # None
    # , yaxis_rangemode = 'tozero'
    xaxis_title=None,
    yaxis_title="Wash Alkalinity",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
)

fig.update_layout(width=1000, height=600)
fig

In [150]:
# group by strip & weighted av of wash_alk_ad
df_strip = (
    df.groupby("pmath_ws")
    .apply(weighted_average, "wash_alk_ad", "wash_qty")
    .reset_index(name="wash_alk_ad")
    # .query('strip not in ("S52","S53","S54","S55")')
)
df_strip
fig = px.bar(
    df_strip,
    x="pmath_ws",
    y="wash_alk_ad",
    title="Wash Alkalinity by Seam",
    template="presentation",
)
fig.add_hline(y=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    yaxis_tickformat=".1%",  # None
    # , yaxis_rangemode = 'tozero'
    xaxis_title=None,
    yaxis_title="Wash Alkalinity",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
)

fig.update_layout(width=1000, height=600)
fig

In [151]:
# group by strip & weighted av of wash_alk_ad
df_strip = (
    df.groupby(["pmath_ws", "coal_type"])
    .apply(weighted_average, "wash_alk_ad", "wash_qty")
    .reset_index(name="wash_alk_ad")
    # .query('strip not in ("S52","S53","S54","S55")')
)
df_strip
fig = px.bar(
    df_strip,
    x="pmath_ws",
    y="wash_alk_ad",
    facet_col="coal_type",
    color="coal_type",
    title="Wash Alkalinity by Seam",
    template="presentation",
)
fig.add_hline(y=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    yaxis_tickformat=".1%",  # None
    # , yaxis_rangemode = 'tozero'
    xaxis_title=None,
    yaxis_title="Wash Alkalinity",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
    showlegend=False,
)
fig.update_xaxes(title_text=None)

fig.update_layout(width=1000, height=600)
fig

In [152]:
# Sort the DataFrame by values
df_sorted = df_hcv.sort_values("wash_alk_ad")

# Calculate the normalized cumulative weights
df_sorted["cumulative_weights"] = df_sorted["wash_qty"].cumsum()
df_sorted["cumulative_weights"] /= df_sorted["cumulative_weights"].iloc[-1]

# Create the ECDF plot
fig = px.line(
    df_sorted,
    x="wash_alk_ad",
    y="cumulative_weights",
    title="High CV Wash Alkalinity ECDF",
    template="presentation",
)
fig.add_vline(x=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    xaxis_tickformat=".1%",  # None
    yaxis_tickformat=".0%",  # None
    # , yaxis_rangemode = 'tozero'
    xaxis_title="Wash Alkalinity",
    yaxis_title="Cumulative Wash Tonnes %",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
    showlegend=False,
)
# fig.update_xaxes(title_text=None)

fig.update_layout(width=1000, height=600)
fig.show()

In [153]:
fig = px.histogram(df_hcv, x="wash_alk_ad", y="wash_qty", histfunc="sum", nbins=50)
fig.add_vline(x=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.show(0)

In [154]:
df_ww = df.query('pmath_ws in ("WW1","WW2","WW3","WW4")')

# plot histograms of wash_alk_ad
fig = px.histogram(
    df_ww, x="wash_alk_ad", y="wash_qty", histfunc="sum", facet_row="pmath_ws", nbins=50
)
fig.add_vline(x=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(width=1000, height=1800)
fig.show(0)

In [155]:
# group by strip & weighted av of wash_alk_ad
df_hcv_seam = (
    df_hcv.groupby("pmath_ws")
    .apply(weighted_average, "wash_alk_ad", "wash_qty")
    .reset_index(name="wash_alk_ad")
    # .query('strip not in ("S52","S53","S54","S55")')
)

# create col 'below_spec' if pmath_ws is in ['MA2','WW1','WW2','WW3'] then 'low_alk_seam' else 'high_alk_seam'
df_hcv_seam["below_spec"] = df_hcv_seam["pmath_ws"].apply(
    lambda x: "low_alk_seam" if x in ["MA2", "WW1", "WW2", "WW3"] else "high_alk_seam"
)

# df_hcv_seam['below_spec'] = df["pmath_ws"].apply(lambda x: 'low_alk_seam' if x in ['MA2','WW1','WW2','WW3'] else "high_alk_seam")

fig = px.bar(
    df_hcv_seam,
    x="pmath_ws",
    y="wash_alk_ad",
    color="below_spec",
    title="Wash Alkalinity by Seam, High CV Coals",
    template="presentation",
)
fig.add_hline(y=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    yaxis_tickformat=".1%",  # None
    # , yaxis_rangemode = 'tozero'
    xaxis_title=None,
    yaxis_title="Wash Alkalinity",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
    showlegend=False,
)

fig.update_layout(width=1000, height=600)
fig
# df_hcv_seam

In [160]:
# group by seam and get the percentage of total wash_qty of each seam from df_hcv
df_hcv_seam_pct = (
    df_hcv.groupby("pmath_ws")
    .apply(lambda x: x["wash_qty"].sum() / df_hcv["wash_qty"].sum())
    .reset_index(name="wash_qty_pct")
)
df_hcv_seam_pct
# grab WW1, WW2, WW3, WW4 and sum the percentages
# df_hcv_seam_pct.query('pmath_ws in ("WW1","WW2","WW3","WW4")')["wash_qty_pct"].sum()

,pmath_ws,wash_qty_pct
0,BR1,0.030404
1,BR2,0.027803
2,BY1,0.027141
3,BY2,0.003810
4,MA2,0.288235
5,PF1,0.045194
6,PF2,0.201810
7,VA1,0.159874
8,VA12,0.135757
9,WW1,0.048320


In [157]:
# get the wash_ash_ad weighted average for each seam
df_hcv_seam_ash = (
    df_hcv.groupby("pmath_ws")
    .apply(weighted_average, "wash_ash_ad", "wash_qty")
    .reset_index(name="wash_ash_ad")
)
df_hcv_seam_ash

,pmath_ws,wash_ash_ad
0,BR1,0.140125
1,BR2,0.145895
2,BY1,0.144197
3,BY2,0.147288
4,MA2,0.140168
5,PF1,0.109475
6,PF2,0.097201
7,VA1,0.097594
8,VA12,0.096878
9,WW1,0.114341


In [158]:
# take df_hcv, create a month field from start_date, filter for MA2, WW1, WW2, WW3, WW4 and by group and sort by start_date_month & seam and calculate the sum of washed qty
df_hcv["start_date_month"] = df_hcv["start_date"].dt.strftime("%Y-%m")
df_hcv_seam_qty = (
    df_hcv.query('pmath_ws in ("WW1","WW2","WW3","WW4")')
    .groupby(["start_date_month", "pmath_ws"])
    .apply(lambda x: x["wash_qty"].sum())
    .reset_index(name="wash_qty")
    .sort_values(["start_date_month", "pmath_ws"])
)

fig = px.bar(
    df_hcv_seam_qty,
    x="start_date_month",
    y="wash_qty",
    color="pmath_ws",
    title="High CV WW Coal Qty by Month, ",
    template="presentation",
)
fig.add_hline(y=0.014, line_width=3, line_dash="dash", line_color="orange")
fig.update_layout(
    yaxis_tickformat=None,  # ".1%",
    # , yaxis_rangemode = 'tozero'
    xaxis_title=None,
    yaxis_title="Washed Quantity (t)",
    # , title='<b>stMATIC Aave V3 Polygon Supply Cap Utilisation</b>'
    hovermode="x",
    # showlegend=False,
)
fig.update_layout(
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, title_text=None)
)

fig.update_layout(width=1000, height=600)
fig

/tmp/ipykernel_701/1727444201.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

